# Intermovie, la bibliothèque du 7ème art

In [1]:
%load_ext autoreload
%autoreload 2

Import des librairies

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functions import split

## Préparation des datasets

In [2]:
##### Création de l'instance de classe #####
splitter_object = split.splitter("./data")

splitter_object.file_splitter("title.principals.tsv", "category", "title.principals")
splitter_object.file_splitter("title.akas", "region", "title.akas")
splitter_object.file_splitter("title.basics", "titleType", "title.basics")

NameError: name 'data_path' is not defined

## Création du premier CSV

In [3]:
#fichier title
data_title1 = pd.read_csv('./data/CURATED/title.basics/movie.csv', sep = ',')
data_title2 = pd.read_csv('./data/CURATED/title.basics/tvMovie.csv', sep = ',')
data_title = pd.concat([data_title1, data_title2], axis = 0)
del data_title1
del data_title2
data_title = data_title.drop(["titleType","originalTitle","isAdult","startYear","endYear","runtimeMinutes","genres"], axis = 1)
data_title.set_index('tconst', inplace  = True)
data_title_clean = data_title.dropna()
del data_title

#fichier noms
data_noms = pd.read_csv('./data/RAW/name.basics.tsv', sep="\t")
data_noms = data_noms.drop(["birthYear", "deathYear"], axis = 1)
data_noms = data_noms.dropna()
data_noms = data_noms[["nconst", "primaryName"]][data_noms['primaryProfession'].str.contains('actor|actress', regex=True)]
data_noms_clean = data_noms.set_index('nconst')
del data_noms

#fichier films
data_films1 = pd.read_csv('./data/CURATED/title.principals/actor.csv', sep=",")
data_films2 = pd.read_csv('./data/CURATED/title.principals/actress.csv', sep=",")
data_films = pd.concat([data_films1, data_films2], axis=0)
del data_films1
del data_films2
data_films = data_films.dropna()
data_films = data_films.drop(["ordering", "job", "characters"], axis = 1)
data_films_clean = data_films[["tconst", "nconst"]][data_films['nconst'].str.contains('n')]
del data_films

#fusion des dataframe
data_films_clean.set_index('tconst', inplace  = True)
data_films_with_title = data_films_clean.join(data_title_clean).reset_index().drop(["tconst"], axis = 1)
del data_films_clean
del data_title_clean
data_films_with_title.set_index('nconst', inplace  = True)
data_final = data_films_with_title.join(data_noms_clean).reset_index().drop(["nconst"], axis = 1)
del data_films_with_title
del data_noms_clean

#convertion en csv
data_final = data_final.dropna()
data_final.to_csv("./data/OUTPUT/csv_req_1.csv")
del data_final

## Création du deuxième CSV

In [4]:
#fichier title
data_title1 = pd.read_csv('./data/CURATED/title.basics/movie.csv', sep = ',')
data_title2 = pd.read_csv('./data/CURATED/title.basics/tvMovie.csv', sep = ',')
data_title = pd.concat([data_title1, data_title2], axis = 0)
del data_title1
del data_title2
data_title = data_title.drop(["titleType","originalTitle","isAdult","startYear","endYear","runtimeMinutes","genres"], axis = 1)
data_title.set_index('tconst', inplace  = True)
data_title_clean = data_title.dropna()
del data_title

#fichier ratings
data_ratings = pd.read_csv('./data/RAW/title.ratings.tsv', sep="\t")
data_ratings = data_ratings.drop(["numVotes"], axis = 1)
data_ratings.set_index('tconst', inplace  = True)
data_ratings_clean = data_ratings.dropna()
del data_ratings

#fichier natio
data_natio = pd.read_csv('./data/CURATED/title.akas/US.csv', sep=",")
data_natio = data_natio.dropna()
data_natio_clean = data_natio.set_index('titleId')
del data_natio

#fusion des dataframe
data_natio_clean = data_natio_clean.join(data_title_clean)
data_final = data_natio_clean.join(data_ratings_clean)
data_final = data_final.drop(["region"], axis = 1)
data_final.drop_duplicates(keep = 'first', inplace=True)
del data_natio_clean

#convertion en csv
data_final.to_csv("./data/OUTPUT/csv_req_2.csv")
del data_final

## Création du troisième CSV

In [5]:
#fichier title
data_title = pd.read_csv('./data/RAW/title.basics.tsv', sep="\t")
data_title = data_title.drop(["titleType","primaryTitle","isAdult","startYear","endYear","runtimeMinutes"], axis = 1)
data_title.set_index('tconst', inplace  = True)
data_title_clean = data_title.dropna()
del data_title

#fichier ratings
data_ratings = pd.read_csv('./data/RAW/title.ratings.tsv', sep="\t")
data_ratings = data_ratings.drop(["numVotes"], axis = 1)
data_ratings.set_index('tconst', inplace  = True)
data_ratings_clean = data_ratings.dropna()
del data_ratings

#fusion des dataframes
df_intermediaire = data_title_clean.join(data_ratings_clean)
df_intermediaire = df_intermediaire.reset_index()
df_intermediaire = df_intermediaire.drop(["tconst"], axis = 1)
df_intermediaire = df_intermediaire.dropna()
del data_title_clean
del data_ratings_clean

#calcul moyenne
df_cat = pd.DataFrame
df_cat = df_intermediaire["genres"].str.split(",", expand=True)
df_intermediaire = df_intermediaire.join(df_cat).drop(["genres"], axis=1)
del df_cat
df_moy1 = df_intermediaire.groupby([0])["averageRating"].mean()
df_moy2 = df_intermediaire.groupby([1])["averageRating"].mean()
df_moy3 = df_intermediaire.groupby([2])["averageRating"].mean()
df_moy = pd.concat([df_moy1, df_moy2, df_moy3], axis = 1, keys = ["moy1", "moy2", "moy3"])
del df_intermediaire
del df_moy1
del df_moy2
del df_moy3
df_moy['moy'] = df_moy[['moy1', 'moy2', 'moy3']].mean(axis=1)
df_moy = df_moy.drop(["moy1", "moy2", "moy3"], axis = 1)

#convertion en csv
df_moy.to_csv('./data/OUTPUT/csv_req_3.csv')
del df_moy

# Création du quatrième CSV

In [6]:
#fichier title
data_title1 = pd.read_csv('./data/CURATED/title.basics/movie.csv', sep = ',')
data_title2 = pd.read_csv('./data/CURATED/title.basics/tvMovie.csv', sep = ',')
data_title = pd.concat([data_title1, data_title2], axis = 0)
del data_title1
del data_title2
data_title = data_title.drop(["titleType","originalTitle","isAdult","startYear","endYear","runtimeMinutes","genres"], axis = 1)
data_title.set_index('tconst', inplace  = True)
data_title_clean = data_title.dropna()
del data_title

#fichier noms
data_noms = pd.read_csv('./data/RAW/name.basics.tsv', sep="\t")
data_noms = data_noms.drop(["birthYear", "deathYear"], axis = 1)
data_noms = data_noms.dropna()
data_noms = data_noms[["nconst", "primaryName"]][data_noms['primaryProfession'].str.contains('actor|actress', regex=True)]
data_noms_clean = data_noms.set_index('nconst')
del data_noms

#fichier films
data_films1 = pd.read_csv('./data/CURATED/title.principals/actor.csv', sep=",")
data_films2 = pd.read_csv('./data/CURATED/title.principals/actress.csv', sep=",")
data_films = pd.concat([data_films1, data_films2], axis=0)
del data_films1
del data_films2
data_films = data_films.dropna()
data_films = data_films.drop(["ordering", "job", "characters"], axis = 1)
data_films_clean = data_films[["tconst", "nconst"]][data_films['nconst'].str.contains('n')]
del data_films

#fichier ratings
data_ratings = pd.read_csv('./data/RAW/title.ratings.tsv', sep="\t")
data_ratings = data_ratings.drop(["numVotes"], axis = 1)
data_ratings.set_index('tconst', inplace  = True)
data_ratings_clean = data_ratings.dropna()
del data_ratings

#fusio, des dataframes
data_films_clean.set_index('tconst', inplace  = True)
data_films_with_title = data_films_clean.join(data_title_clean)
data_films_with_title = data_films_with_title.join(data_ratings_clean).reset_index().drop(["tconst"], axis = 1)
del data_films_clean
del data_title_clean
del data_ratings_clean
data_films_with_title.set_index('nconst', inplace  = True)
data_final = data_films_with_title.join(data_noms_clean).reset_index().drop(["nconst"], axis = 1)
del data_films_with_title
del data_noms_clean

#calcul de la moyenne
data_final = data_final.dropna()
data_final = data_final.groupby(["primaryName"]).mean()

#conversion en csv
data_final.to_csv('./data/OUTPUT/csv_req_4.csv')
del data_final